In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("C:\\Users\\Sai Teja\\Downloads\\0000000000002747_training_twitter_x_y_train.csv")
df = df.drop(columns=['tweet_id','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'])
df

,airline_sentiment,text
0,negative,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,@SouthwestAir seeing your workers time in and ...
2,positive,@united Flew ORD to Miami and back and had gr...
3,negative,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,negative,@united so our flight into ORD was delayed bec...
...,...,...
10975,neutral,@AmericanAir followback
10976,positive,@united thanks for the help. Wish the phone re...
10977,negative,@usairways the. Worst. Ever. #dca #customerser...
10978,negative,@nrhodes85: look! Another apology. DO NOT FLY ...


In [3]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [4]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [5]:
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

In [6]:
from nltk import pos_tag
def cleanTweet(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [7]:
tweets = []
category = []
for i in range(len(df)):
    tweets.append([df['text'][i]])
    category.append(df['airline_sentiment'][i])

In [8]:
for i in range(len(tweets)):
    tweets[i] = tweets[i][0].split(" ")

In [9]:
import nltk
tweets = [cleanTweet(tweet) for tweet in tweets]

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
text_tweets = [" ".join(tweet) for tweet in tweets]

In [12]:
count_vec = CountVectorizer(max_features = 2000)
tweets_features = count_vec.fit_transform(text_tweets)
tweets_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
df1 = pd.read_csv("C:\\Users\\Sai Teja\\Downloads\\0000000000002747_test_twitter_x_test.csv")
df1 = df1.drop(columns=['tweet_id','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],axis=1)
x_test = []
for i in range(len(df1)):
    x_test.append(df1['text'][i])
x_test_features = count_vec.transform(x_test)

In [17]:
from sklearn.svm import SVC

In [24]:
svc = SVC()
svc.fit(tweets_features, category)

SVC()

In [35]:
predictions = svc.predict(x_test_features)
predictions = predictions.tolist()

list

In [38]:
predictions = [[i] for i in predictions]

In [42]:
import csv

with open('Predictions.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    
    for row in predictions:
        writer.writerow(row)

In [43]:
svc.score(tweets_features, category)

0.9045537340619307